In [14]:
import numpy as np
import scipy.io
from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
from scipy.signal import butter, lfilter
from scipy.signal import butter, sosfilt, sosfreqz
from scipy.signal import welch
from scipy import signal
import os
import pandas as pd

In [15]:
import matplotlib.pyplot as plt

In [16]:
save_path = 'D:\EECE499\Features\EEGFeatures_FixedWindow.xlsx'
#save_path = '..\..\..\EEGFeatures_FixedWindow.xlsx'

In [17]:
sample_rate = 32

In [18]:
max_length = 7054

In [19]:
7054 / 590

11.95593220338983

In [20]:
window = 590

In [21]:
window_intervals_even = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])*window #12
window_intervals_odd = np.array([0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5, 10.5, 11.5])*window #11

In [22]:
window_intervals_even

array([ 590, 1180, 1770, 2360, 2950, 3540, 4130, 4720, 5310, 5900, 6490])

In [23]:
window_intervals_odd = window_intervals_odd.astype(int)
window_intervals_odd

array([ 295,  885, 1475, 2065, 2655, 3245, 3835, 4425, 5015, 5605, 6195,
       6785])

In [24]:
yup = np.array(list(range(1, 7054)))

In [25]:
for i in range(1, 11):
    print(yup[window_intervals_even[i - 1]:window_intervals_even[i]].shape[0] == window)
    
for i in range(1, 12):
    print(yup[window_intervals_odd[i - 1]:window_intervals_odd[i]].shape[0] == window)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [26]:
data_frames = []

for user_id in range(1, 59):
    print('User ID: ' + str(user_id))
    
    neuro_sky_EEG = {
        'avg_first_deriv' : [[] for i in range(23)],
        'percent_neg_deriv' : [[] for i in range(23)],
        'mean_peaks' : [[] for i in range(23)],
        'avg_first_deriv_inv' : [[] for i in range(23)],
        'mean_peaks_inv' : [[] for i in range(23)],
        'mean' : [[] for i in range(23)],
        'std' : [[] for i in range(23)],
        'skewness' : [[] for i in range(23)],
        'kurtosis' : [[] for i in range(23)],
        'percent_over_std' : [[] for i in range(23)],
        'percent_under_std' : [[] for i in range(23)]
    }
    attention = {
        'avg_first_deriv' : [[] for i in range(23)],
        'percent_neg_deriv' : [[] for i in range(23)],
        'mean_peaks' : [[] for i in range(23)],
        'avg_first_deriv_inv' : [[] for i in range(23)],
        'mean_peaks_inv' : [[] for i in range(23)],
        'mean' : [[] for i in range(23)],
        'std' : [[] for i in range(23)],
        'skewness' : [[] for i in range(23)],
        'kurtosis' : [[] for i in range(23)],
        'percent_over_std' : [[] for i in range(23)],
        'percent_under_std' : [[] for i in range(23)]
    }
    meditation = {
        'avg_first_deriv' : [[] for i in range(23)],
        'percent_neg_deriv' : [[] for i in range(23)],
        'mean_peaks' : [[] for i in range(23)],
        'avg_first_deriv_inv' : [[] for i in range(23)],
        'mean_peaks_inv' : [[] for i in range(23)],
        'mean' : [[] for i in range(23)],
        'std' : [[] for i in range(23)],
        'skewness' : [[] for i in range(23)],
        'kurtosis' : [[] for i in range(23)],
        'percent_over_std' : [[] for i in range(23)],
        'percent_under_std' : [[] for i in range(23)]
    }
    alpha = {
        'avg_first_deriv' : [[] for i in range(23)],
        'percent_neg_deriv' : [[] for i in range(23)],
        'mean_peaks' : [[] for i in range(23)],
        'avg_first_deriv_inv' : [[] for i in range(23)],
        'mean_peaks_inv' : [[] for i in range(23)],
        'mean' : [[] for i in range(23)],
        'std' : [[] for i in range(23)],
        'skewness' : [[] for i in range(23)],
        'kurtosis' : [[] for i in range(23)],
        'percent_over_std' : [[] for i in range(23)],
        'percent_under_std' : [[] for i in range(23)]
    }
    beta = {
        'avg_first_deriv' : [[] for i in range(23)],
        'percent_neg_deriv' : [[] for i in range(23)],
        'mean_peaks' : [[] for i in range(23)],
        'avg_first_deriv_inv' : [[] for i in range(23)],
        'mean_peaks_inv' : [[] for i in range(23)],
        'mean' : [[] for i in range(23)],
        'std' : [[] for i in range(23)],
        'skewness' : [[] for i in range(23)],
        'kurtosis' : [[] for i in range(23)],
        'percent_over_std' : [[] for i in range(23)],
        'percent_under_std' : [[] for i in range(23)]
    }
    delta = {
        'avg_first_deriv' : [[] for i in range(23)],
        'percent_neg_deriv' : [[] for i in range(23)],
        'mean_peaks' : [[] for i in range(23)],
        'avg_first_deriv_inv' : [[] for i in range(23)],
        'mean_peaks_inv' : [[] for i in range(23)],
        'mean' : [[] for i in range(23)],
        'std' : [[] for i in range(23)],
        'skewness' : [[] for i in range(23)],
        'kurtosis' : [[] for i in range(23)],
        'percent_over_std' : [[] for i in range(23)],
        'percent_under_std' : [[] for i in range(23)]
    }
    gamma = {
        'avg_first_deriv' : [[] for i in range(23)],
        'percent_neg_deriv' : [[] for i in range(23)],
        'mean_peaks' : [[] for i in range(23)],
        'avg_first_deriv_inv' : [[] for i in range(23)],
        'mean_peaks_inv' : [[] for i in range(23)],
        'mean' : [[] for i in range(23)],
        'std' : [[] for i in range(23)],
        'skewness' : [[] for i in range(23)],
        'kurtosis' : [[] for i in range(23)],
        'percent_over_std' : [[] for i in range(23)],
        'percent_under_std' : [[] for i in range(23)]
    }
    theta = {
        'avg_first_deriv' : [[] for i in range(23)],
        'percent_neg_deriv' : [[] for i in range(23)],
        'mean_peaks' : [[] for i in range(23)],
        'avg_first_deriv_inv' : [[] for i in range(23)],
        'mean_peaks_inv' : [[] for i in range(23)],
        'mean' : [[] for i in range(23)],
        'std' : [[] for i in range(23)],
        'skewness' : [[] for i in range(23)],
        'kurtosis' : [[] for i in range(23)],
        'percent_over_std' : [[] for i in range(23)],
        'percent_under_std' : [[] for i in range(23)]
    }
    
    presentation_id = []
    
    channels = [neuro_sky_EEG, attention, meditation, alpha, beta, delta, gamma, theta]
    channel_names = ['neuro_sky', 'attention', 'meditation', 'alpha', 'beta', 'delta', 'gamma', 'theta']
    
    for clip_id in range(1, 37):
        #print('Clip ID: ' + str(clip_id))
        
        data_path = 'D:/EECE499/Raw/MyECGFunc/ASCERTAIN_Raw/EEGData/Movie_P' + str(user_id).zfill(2) + '/EEG_Clip' + str(clip_id) + '.mat'    
        #data_path = './../../ASCERTAIN_Raw/EEGData/Movie_P' + str(user_id).zfill(2) + '/EEG_Clip' + str(clip_id) + '.mat'
        
        presentation_id.append(str(user_id).zfill(2) + str(clip_id).zfill(2))
        
        signal = scipy.io.loadmat(data_path)
        neuro_sky_EEG_raw = np.array(signal['ThisEEG'][0])
        attention_raw = np.array(signal['ThisEEG'][1])
        meditation_raw = np.array(signal['ThisEEG'][2])
        alpha_raw = np.array(signal['ThisEEG'][3])
        beta_raw = np.array(signal['ThisEEG'][4])
        delta_raw = np.array(signal['ThisEEG'][5])
        gamma_raw = np.array(signal['ThisEEG'][6])
        theta_raw = np.array(signal['ThisEEG'][7])
        
        w_ = 51 #101
        order = 4 #3
        try:
            neuro_sky_EEG_raw = savgol_filter(neuro_sky_EEG_raw, w_, order)
            attention_raw = savgol_filter(attention_raw, w_, order)
            meditation_raw = savgol_filter(meditation_raw, w_, order)
            alpha_raw = savgol_filter(alpha_raw, w_, order)
            beta_raw = savgol_filter(beta_raw, w_, order)
            delta_raw = savgol_filter(delta_raw, w_, order)
            gamma_raw = savgol_filter(gamma_raw, w_, order)
            theta_raw = savgol_filter(theta_raw, w_, order)
        except:
            l = len(neuro_sky_EEG_raw)
            neuro_sky_EEG_raw = np.zeros(l)
            attention_raw = np.zeros(l)
            meditation_raw = np.zeros(l)
            alpha_raw = np.zeros(l)
            beta_raw = np.zeros(l)
            delta_raw = np.zeros(l)
            gamma_raw = np.zeros(l)
            theta_raw = np.zeros(l)
            
        if (max_length > neuro_sky_EEG_raw.shape[0]):
            neuro_sky_EEG_raw = np.pad(neuro_sky_EEG_raw, (0, max_length - neuro_sky_EEG_raw.shape[0]), 'mean')
            attention_raw = np.pad(attention_raw, (0, max_length - attention_raw.shape[0]), 'mean') 
            meditation_raw = np.pad(meditation_raw, (0, max_length - meditation_raw.shape[0]), 'mean')  
            alpha_raw = np.pad(alpha_raw, (0, max_length - alpha_raw.shape[0]), 'mean') 
            beta_raw = np.pad(beta_raw, (0, max_length - beta_raw.shape[0]), 'mean') 
            delta_raw = np.pad(delta_raw, (0, max_length - delta_raw.shape[0]), 'mean') 
            gamma_raw = np.pad(gamma_raw, (0, max_length - gamma_raw.shape[0]), 'mean') 
            theta_raw = np.pad(theta_raw, (0, max_length - theta_raw.shape[0]), 'mean') 
        else:
            neuro_sky_EEG_raw = neuro_sky_EEG_raw[:max_length]
            attention_raw = attention_raw[:max_length]
            meditation_raw = meditation_raw[:max_length]
            alpha_raw = alpha_raw[:max_length]
            beta_raw = beta_raw[:max_length]
            delta_raw = delta_raw[:max_length]
            gamma_raw = gamma_raw[:max_length]
            theta_raw = theta_raw[:max_length]
            
        signals = [neuro_sky_EEG_raw, attention_raw, meditation_raw, alpha_raw, beta_raw, delta_raw, gamma_raw, theta_raw]
            
        for channel, signal in zip(channels, signals):
            
            channel['avg_first_deriv'][0].append(np.mean(np.diff(signal[:window_intervals_even[0]])))
            for i in range(1, 11):
                channel['avg_first_deriv'][i].append(np.mean(np.diff(signal[window_intervals_even[i - 1]:window_intervals_even[i]])))
            channel['avg_first_deriv'][11].append(np.mean(np.diff(signal[window_intervals_even[10]:])))
            for i in range(1, 12):
                channel['avg_first_deriv'][i + 11].append(np.mean(np.diff(signal[window_intervals_odd[i - 1]:window_intervals_odd[i]])))
                
                
            deriv = np.diff(signal[:window_intervals_even[0]])
            channel['percent_neg_deriv'][0].append(len(deriv[deriv < 0]) / len(deriv))
            for i in range(1, 11):
                deriv = np.diff(signal[window_intervals_even[i - 1]:window_intervals_even[i]])
                channel['percent_neg_deriv'][i].append(len(deriv[deriv < 0]) / len(deriv)) 
            deriv = np.diff(signal[window_intervals_even[10]:])
            channel['percent_neg_deriv'][11].append(len(deriv[deriv < 0]) / len(deriv))
            for i in range(1, 12):
                deriv = np.diff(signal[window_intervals_odd[i - 1]:window_intervals_odd[i]])
                channel['percent_neg_deriv'][i + 11].append(len(deriv[deriv < 0]) / len(deriv))
            
            
            peaks, _ = find_peaks(signal[:window_intervals_even[0]])
            channel['mean_peaks'][0].append(len(peaks) / len(signal))
            for i in range(1, 11):
                peaks, _ = find_peaks(signal[window_intervals_even[i - 1]:window_intervals_even[i]])
                channel['mean_peaks'][i].append(len(peaks) / len(signal)) 
            peaks, _ = find_peaks(signal[window_intervals_even[10]:])
            channel['mean_peaks'][11].append(len(peaks) / len(signal))
            for i in range(1, 12):
                peaks, _ = find_peaks(signal[window_intervals_odd[i - 1]:window_intervals_odd[i]])
                channel['mean_peaks'][i + 11].append(len(peaks) / len(signal))
                
                
            channel['avg_first_deriv_inv'][0].append(np.mean(np.diff(1 / signal[:window_intervals_even[0]])))
            for i in range(1, 11):
                channel['avg_first_deriv_inv'][i].append(np.mean(np.diff(1 / signal[window_intervals_even[i - 1]:window_intervals_even[i]])))
            channel['avg_first_deriv_inv'][11].append(np.mean(np.diff(1 / signal[window_intervals_even[10]:])))
            for i in range(1, 12):
                channel['avg_first_deriv_inv'][i + 11].append(np.mean(np.diff(1 / signal[window_intervals_odd[i - 1]:window_intervals_odd[i]])))
              
            
            peaks, _ = find_peaks(1 / signal[:window_intervals_even[0]])
            channel['mean_peaks_inv'][0].append(len(peaks) / len(signal))
            for i in range(1, 11):
                peaks, _ = find_peaks(1 / signal[window_intervals_even[i - 1]:window_intervals_even[i]])
                channel['mean_peaks_inv'][i].append(len(peaks) / len(signal)) 
            peaks, _ = find_peaks(1 / signal[window_intervals_even[10]:])
            channel['mean_peaks_inv'][11].append(len(peaks) / len(signal))
            for i in range(1, 12):
                peaks, _ = find_peaks(1 / signal[window_intervals_odd[i - 1]:window_intervals_odd[i]])
                channel['mean_peaks_inv'][i + 11].append(len(peaks) / len(signal))
            
            
            channel['mean'][0].append(np.mean(signal[:window_intervals_even[0]]))
            for i in range(1, 11):
                channel['mean'][i].append(np.mean(signal[window_intervals_even[i - 1]:window_intervals_even[i]]))
            channel['mean'][11].append(np.mean(signal[window_intervals_even[10]:]))
            for i in range(1, 12):
                channel['mean'][i + 11].append(np.mean(signal[window_intervals_odd[i - 1]:window_intervals_odd[i]]))
              
            
            channel['std'][0].append(np.std(signal[:window_intervals_even[0]]))
            for i in range(1, 11):
                channel['std'][i].append(np.std(signal[window_intervals_even[i - 1]:window_intervals_even[i]]))
            channel['std'][11].append(np.std(signal[window_intervals_even[10]:]))
            for i in range(1, 12):
                channel['std'][i + 11].append(np.std(signal[window_intervals_odd[i - 1]:window_intervals_odd[i]]))
            
            
            channel['skewness'][0].append(skew(signal[:window_intervals_even[0]]))
            for i in range(1, 11):
                channel['skewness'][i].append(skew(signal[window_intervals_even[i - 1]:window_intervals_even[i]]))
            channel['skewness'][11].append(skew(signal[window_intervals_even[10]:]))
            for i in range(1, 12):
                channel['skewness'][i + 11].append(skew(signal[window_intervals_odd[i - 1]:window_intervals_odd[i]]))
                
            
            channel['kurtosis'][0].append(kurtosis(signal[:window_intervals_even[0]]))
            for i in range(1, 11):
                channel['kurtosis'][i].append(kurtosis(signal[window_intervals_even[i - 1]:window_intervals_even[i]]))
            channel['kurtosis'][11].append(kurtosis(signal[window_intervals_even[10]:]))
            for i in range(1, 12):
                channel['kurtosis'][i + 11].append(kurtosis(signal[window_intervals_odd[i - 1]:window_intervals_odd[i]]))
                
                
            std_ = np.std(signal[:window_intervals_even[0]])
            signal_ = signal[:window_intervals_even[0]]
            channel['percent_over_std'][0].append(len(signal_[signal_ > np.mean(signal_) + std_]) / len(signal_))
            for i in range(1, 11):
                std_ = np.std(signal[window_intervals_even[i - 1]:window_intervals_even[i]])
                signal_ = signal[window_intervals_even[i - 1]:window_intervals_even[i]]
                channel['percent_over_std'][i].append(len(signal_[signal_ > np.mean(signal_) + std_]) / len(signal_))
            std_ = np.std(signal[window_intervals_even[10]:])
            signal_ = signal[window_intervals_even[10]:]
            channel['percent_over_std'][11].append(len(signal_[signal_ > np.mean(signal_) + std_]) / len(signal_))
            for i in range(1, 12):
                std_ = np.std(signal[window_intervals_odd[i - 1]:window_intervals_odd[i]])
                signal_ = signal[window_intervals_odd[i - 1]:window_intervals_odd[i]]
                channel['percent_over_std'][i + 11].append(len(signal_[signal_ > np.mean(signal_) + std_]) / len(signal_))
            
            
            std_ = np.std(signal[:window_intervals_even[0]])
            signal_ = signal[:window_intervals_even[0]]
            channel['percent_under_std'][0].append(len(signal_[signal_ < np.mean(signal_) + std_]) / len(signal_))
            for i in range(1, 11):
                std_ = np.std(signal[window_intervals_even[i - 1]:window_intervals_even[i]])
                signal_ = signal[window_intervals_even[i - 1]:window_intervals_even[i]]
                channel['percent_under_std'][i].append(len(signal_[signal_ < np.mean(signal_) + std_]) / len(signal_))
            std_ = np.std(signal[window_intervals_even[10]:])
            signal_ = signal[window_intervals_even[10]:]
            channel['percent_under_std'][11].append(len(signal_[signal_ < np.mean(signal_) + std_]) / len(signal_))
            for i in range(1, 12):
                std_ = np.std(signal[window_intervals_odd[i - 1]:window_intervals_odd[i]])
                signal_ = signal[window_intervals_odd[i - 1]:window_intervals_odd[i]]
                channel['percent_under_std'][i + 11].append(len(signal_[signal_ < np.mean(signal_) + std_]) / len(signal_))
        
        #plt.plot(neuro_sky_EEG_raw)
        #plt.show()
        
    user_data = pd.DataFrame({
        'presentation_id' : presentation_id
    })
    
    for channel, channel_name in zip(channels, channel_names):
        for i in range(23):
            user_data[channel_name + '_avg_first_deriv_' + str(i)] = channel['avg_first_deriv'][i]
            user_data[channel_name + '_percent_neg_deriv_' + str(i)] = channel['percent_neg_deriv'][i]
            user_data[channel_name + '_mean_peaks_' + str(i)] = channel['mean_peaks'][i]
            user_data[channel_name + '_mean_' + str(i)] = channel['mean'][i]
            user_data[channel_name + '_std_' + str(i)] = channel['std'][i]
            user_data[channel_name + '_mean_peaks_inv_' + str(i)] = channel['mean_peaks_inv'][i]
            user_data[channel_name + '_avg_first_deriv_inv_' + str(i)] = channel['avg_first_deriv_inv'][i]
            user_data[channel_name + '_skewness_' + str(i)] = channel['skewness'][i]
            user_data[channel_name + '_kurtosis_' + str(i)] = channel['kurtosis'][i]
            user_data[channel_name + '_percent_over_std_' + str(i)] = channel['percent_over_std'][i]
            user_data[channel_name + '_percent_under_std_' + str(i)] = channel['percent_under_std'][i]
    
    data_frames.append(user_data)


User ID: 1


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:191: RuntimeWarning: invalid value encountered in less
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:193: RuntimeWarning: invalid value encountered in less
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:196: RuntimeWarning: invalid value encountered in less
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:269: RuntimeWarning: invalid value encountered in greater
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:272: RuntimeWarning: invalid value encountered in greater
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:276: RuntimeWarning: invalid value encountered in greater
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:285: RuntimeWarning: invalid value encountered in less
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:288: RuntimeWarning: invalid value encountered in less
C:\ProgramData\Anaconda3\lib\si

User ID: 2


C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:188: RuntimeWarning: invalid value encountered in less
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:265: RuntimeWarning: invalid value encountered in greater
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:281: RuntimeWarning: invalid value encountered in less
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)


User ID: 3


...

User ID: 55


C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may b

User ID: 56
User ID: 57


C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may be poorly conditioned
  xx_edge, polyorder)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_savitzky_golay.py:187: RankWarning: Polyfit may b

User ID: 58


In [27]:
EEG_Features_FixedWindow = pd.concat(data_frames,ignore_index=True)

In [28]:
EEG_Features_FixedWindow

,presentation_id,neuro_sky_avg_first_deriv_0,neuro_sky_percent_neg_deriv_0,neuro_sky_mean_peaks_0,neuro_sky_mean_0,neuro_sky_std_0,neuro_sky_mean_peaks_inv_0,neuro_sky_avg_first_deriv_inv_0,neuro_sky_skewness_0,neuro_sky_kurtosis_0,...,theta_percent_neg_deriv_22,theta_mean_peaks_22,theta_mean_22,theta_std_22,theta_mean_peaks_inv_22,theta_avg_first_deriv_inv_22,theta_skewness_22,theta_kurtosis_22,theta_percent_over_std_22,theta_percent_under_std_22
0,0101,0.098397,0.488964,0.005103,318.469850,50.639413,0.005245,-9.898546e-07,-1.669670,5.214824,...,0.000000,0.000000,154.853318,2.842171e-14,0.000000,0.000000e+00,1.000000,-2.000000,0.000000,0.000000
1,0102,-0.169634,0.507640,0.005812,323.358846,37.553478,0.005812,1.258245e-06,0.133364,0.839917,...,0.000000,0.000000,104.597244,1.421085e-14,0.000000,0.000000e+00,1.000000,-2.000000,0.000000,0.000000
2,0103,0.106091,0.517827,0.005387,323.622821,23.909361,0.005387,-1.036484e-06,0.046240,-0.196284,...,0.000000,0.000000,41.658683,7.105427e-15,0.000000,0.000000e+00,-1.000000,-2.000000,0.000000,1.000000
3,0104,0.007632,0.521222,0.003261,336.826588,55.145714,0.003119,-7.026531e-08,0.804942,2.391952,...,0.000000,0.000000,82.669052,1.421085e-14,0.000000,0.000000e+00,1.000000,-2.000000,0.000000,0.000000
4,0105,-0.047983,0.488964,0.005529,359.046825,95.083075,0.005387,5.910492e-07,0.218533,-0.219058,...,0.000000,0.000000,550.077516,0.000000e+00,0.000000,0.000000e+00,0.000000,-3.000000,0.000000,0.000000
5,0106,0.236016,0.451613,0.004111,330.219892,62.858457,0.003969,-2.302911e-06,0.417045,2.506852,...,0.000000,0.000000,529.457110,0.000000e+00,0.000000,0.000000e+00,0.000000,-3.000000,0.000000,0.000000
6,0107,-0.081079,0.512733,0.004395,339.887914,53.173971,0.004536,7.498750e-07,1.169207,1.746189,...,0.000000,0.000000,216.701065,5.684342e-14,0.000000,0.000000e+00,-1.000000,-2.000000,0.000000,1.000000
7,0108,0.003382,0.482173,0.003544,334.096597,48.324107,0.003544,-2.867674e-08,1.480709,4.902085,...,0.000000,0.000000,13.388698,3.552714e-15,0.000000,0.000000e+00,1.000000,-2.000000,0.000000,0.000000
8,0109,-0.018513,0.500849,0.005103,320.639900,30.351694,0.005103,1.927517e-07,2.020635,7.933553,...,0.000000,0.000000,156.268805,2.842171e-14,0.000000,0.000000e+00,1.000000,-2.000000,0.000000,0.000000
9,0110,-0.186079,0.490662,0.003402,306.114269,57.909406,0.003261,1.617298e-06,-2.190584,6.816807,...,0.000000,0.000000,143.500000,2.842171e-14,0.000000,0.000000e+00,1.000000,-2.000000,0.000000,0.000000


In [29]:
EEG_Features_FixedWindow.to_excel(save_path)